In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import pickle as pkl
import glob
import numpy as np
import cv2
from img_load import load_bimg
from NPSmethods2 import *

In [ ]:
psat=277

#Settings of imaging system
imgSysData = { 
    "CCDPixelSize": 13,      # pixel size of the CCD, in micron 
    "magnification": 27,      # 799.943 / 29.9099, # magnification of the imaging system 
    "wavelen"     : 0.852,     # wavelength of the imaging beam, in micron 
    "NA"          : 0.37,      # numerical aperture of the objective 
    "ODtoAtom"    : 13
}

#Configurations for NPSMethods (here we only need do_Fit)
choices = {
    "if_Save"                     : True,
    "do_Fit"                      : True,
    "plot_2dGas"                  : False,
    "plot_NoisePowSpec"           : True,
    "plot_NoisePowSpec_LineCut"   : True,
    "plot_Pupil"                  : False,
    "plot_PSF"                    : False,
    "plot_PSF_LineCut"            : False,
    "plot_PSF_abs2"               : False,
    "plot_PSF_abs2_LineCut"       : False,
    "plot_Sk"                     : False,
    "normalize"                   : False
}

#Directory containing image folders
path = r'C:\Users\eric0\Jupyter Notebooks\Cold Physics\raw_image'

#Directory where ML data will output to
resDir = r'C:\Users\eric0\Jupyter Notebooks\Cold Physics\results'

trapRegion = (slice(100, 200), slice(100, 200))
    # the region where the atoms located, [(xmin, xmax), (ymin, ymax)] (pixel)

noiseRegion = (slice(0, 300), slice(0, 300))
    # the region chosen for noise analysis, [(xmin, xmax), (ymin, ymax)] (pixel)

numOfImgsInEachRun = 50      # number of images for each run of analysis

rampingParameter = 0

#Lists where OD images and parameters will be stored
rawimages = []
parameters = []
bkgMax = 0
bkgMin = 100

#Sort through folders in path directory for raw images, extract OD image and parameters, then append to lists
for folder in glob.iglob(path + "\\*"):
    result=load_bimg(folder + "\\", raw=True)
    odimg=-np.log(result['raw1']/result['raw2'])-(result['raw1']-result['raw2'])/psat
    _, _, _, _, popt, _, bkg = doCalibration(folder, resDir, trapRegion, noiseRegion, numOfImgsInEachRun, rampingParameter, imgSysData, choices)
    print(popt)
    print(bkg)
    if bkg > bkgMax:
        bkgMax = bkg
    if bkg < bkgMin:
        bkgMin = bkg
    for i in range(len(odimg)):
        rawimages.append(odimg[i])
        parameters.append(popt)


In [ ]:
#Sort through parameters and find the maximum and minimum for each separate parameter
seedParasMax = np.empty_like(parameters[0])
seedParasMin = np.empty_like(parameters[0])
rotparam = np.array(parameters).T
for i in range(len(parameters[0])):
    seedParasMax[i] = np.max(rotparam[i])
    seedParasMin[i] = np.min(rotparam[i])

In [ ]:
#Check that values look reasonable
print(seedParasMax)
print(seedParasMin)

In [ ]:
#Save as pickle files
pkl.dump(seedParasMin, open(resDir + "\\seedParasMin.pkl", "wb"))
pkl.dump(seedParasMax, open(resDir + "\\seedParasMax.pkl", "wb"))
pkl.dump(bkgMax, open(resDir + "\\bkgMax.pkl", "wb"))
pkl.dump(bkgMin, open(resDir + "\\bkgMin.pkl", "wb"))